In [ ]:
#%%
import time
from optibook.synchronous_client import Exchange
import logging

logger = logging.getLogger('client')
logger.setLevel('INFO')

exchange = Exchange()
exchange.connect()

#%%

# Define the specific pairs of original and dual-listed stocks
stock_pairs = [
    ('NVDA', 'NVDA_DUAL'),
    ('SAN', 'SAN_DUAL')
]

#%%
# Initialize dictionary to track positions dynamically
def initialize_positions1():
    positions = {}
    for original_stock, dual_stock in stock_pairs:
        positions[dual_stock] = exchange.get_positions()[dual_stock]
        positions[original_stock] = exchange.get_positions()[dual_stock]

    return positions
initialize_positions1()

#%%
def initialize_positions():
    positions = {}
    for _, dual_stock in stock_pairs:
        positions[dual_stock] = exchange.get_positions()[dual_stock]
        logger.info(f"Initial position for {dual_stock}: {positions[dual_stock]}")
    return positions

def get_best_prices(stock):
    book = exchange.get_last_price_book(stock)
    return book.bids[0].price if book.bids else None, book.asks[0].price if book.asks else None

#%%
def arbitraging(stock_pair):
    original_stock, dual_stock = stock_pair
    orig_bid, orig_ask = get_best_prices(original_stock)
    dual_bid, dual_ask = get_best_prices(dual_stock)

    current_positions = exchange.get_positions()
    profit_threshold = 0.05  # Minimum profit to initiate a trade
    # Buy dual at ask price and sell original at bid price
    if dual_ask and orig_bid and (orig_bid - dual_ask) > profit_threshold:
        profit = orig_bid - dual_ask
        if profit > (profit_threshold):
            buy_quantity = 100 - exchange.get_positions()[dual_stock]
            sell_quantity = exchange.get_positions()[original_stock] + 100

            if buy_quantity > 0 and sell_quantity > 0:
                print("1", profit)
                # Increase the buy price slightly to make sure the order goes through
                buy_price = dual_ask + profit/2.1
                exchange.insert_order(dual_stock, price=buy_price, volume=buy_quantity, side='bid', order_type='ioc')
                new_position = exchange.get_positions()
                if new_position != current_positions:
                    logger.info(f"Arbitrage. Bought {new_position[dual_stock] - current_positions[dual_stock]} of {dual_stock} at {buy_price} and {profit} profit")
                    current_positions = exchange.get_positions()
                    print("1.5", profit)
                    # Decrease the sell price slightly to make sure the order goes through
                    sell_price = orig_bid - profit/2.1
                    exchange.insert_order(original_stock, price=sell_price, volume=sell_quantity, side='ask', order_type='ioc')
                    new_position = exchange.get_positions()
                    if new_position != current_positions:
                        logger.info(f"Arbitrage. Sold {new_position[original_stock] - current_positions[original_stock]} of {original_stock} at {sell_price} and {profit} profit")
                        current_positions = exchange.get_positions()
                
    # Buy original at ask price and sell dual at bid price
    if orig_ask and dual_bid and (dual_bid - orig_ask) > (profit_threshold):
        profit = dual_bid - orig_ask
        if profit > (profit_threshold):
            buy_quantity = 100 - exchange.get_positions()[original_stock]
            sell_quantity = exchange.get_positions()[dual_stock] + 100

            if buy_quantity > 0 and sell_quantity > 0: 
                print("2", profit)
                # Decrease the buy price slightly to make sure the order goes through
                buy_price = orig_ask + profit/2.1
                exchange.insert_order(original_stock, price=buy_price, volume=buy_quantity, side='bid', order_type='ioc')
                new_position = exchange.get_positions()
                if new_position != current_positions:
                    logger.info(f"Arbitrage. Bought {new_position[original_stock] - current_positions[original_stock]} of {original_stock} at {buy_price} and {profit} profit")
                    current_positions = exchange.get_positions()
                    print("2.5", profit)
                    # Increase the sell price slightly to make sure the order goes through
                    sell_price = dual_bid - profit/2.1
                    exchange.insert_order(dual_stock, price=sell_price, volume=sell_quantity, side='ask', order_type='ioc')
                    new_position = exchange.get_positions()
                    if new_position != current_positions:
                        logger.info(f"Arbitrage. Sold {new_position[dual_stock] - current_positions[dual_stock]} of {dual_stock} at {sell_price} and {profit} profit")
                        current_positions = exchange.get_positions()

                

#%%

def hedge(stock_pair, last_bid, last_ask):
        og_stock, dual_stock = stock_pair
        
        # Check number of positions we have in quoting and hedging instrument
        positions = exchange.get_positions()
        current = positions[og_stock]

        order_book = exchange.get_last_price_book(instrument_id=og_stock)

        # If there are no bids/asks, we set the bid/ask price to the previous bid/ask price
        # If there are bids/asks, we update our bid/ask price to the current best bid/ask price
        if not order_book.bids:
            best_bid = last_bid[og_stock]
            if og_stock not in bid_price_history:
                bid_price_history[og_stock] = [last_bid[og_stock]]  # Ensure it's a list
            else:
                bid_price_history[og_stock].append(last_bid[og_stock])
        else:
            if og_stock not in bid_price_history:
                bid_price_history[og_stock] = [order_book.bids[0].price]  # Ensure it's a list
            else:
                bid_price_history[og_stock].append(order_book.bids[0].price)
            best_bid = sum(bid_price_history[og_stock]) / len(bid_price_history[og_stock]) - 0.05

        if len(bid_price_history[og_stock]) > HISTORY:
            del bid_price_history[og_stock][0]

        if not order_book.asks:
            best_ask = last_ask[og_stock]
            if og_stock not in ask_price_history:
                ask_price_history[og_stock] = [last_ask[og_stock]]  # Ensure it's a list
            else:
                ask_price_history[og_stock].append(last_ask[og_stock])
        else:
            if og_stock not in ask_price_history:
                ask_price_history[og_stock] = [order_book.asks[0].price]  # Ensure it's a list
            else:
                ask_price_history[og_stock].append(order_book.asks[0].price)

            best_ask = sum(ask_price_history[og_stock]) / len(ask_price_history[og_stock]) + 0.05

        if len(ask_price_history[og_stock]) > HISTORY:
            del ask_price_history[og_stock][0]

        max_ask_volume = max(0, positions[og_stock] + 100)
        max_bid_volume = max(0, 100 - positions[og_stock])

        # If we are long overall, we need to sell in the hedging exchange
        if max_ask_volume > 0:
            if best_ask > last_bid[og_stock]:
                exchange.insert_order(og_stock, price=best_ask, volume=max_ask_volume, side="ask", order_type="ioc")
                new = exchange.get_positions()
                time.sleep(0.04)
                if current != new[og_stock]:
                    last_ask[og_stock] = best_ask # was positionTotal
                    logger.info(f"Hedge. IOC Ask of {new[og_stock] - current} of {og_stock} at {best_ask}")
                    current = new[og_stock]
        if max_bid_volume > 0:
            if best_bid < (last_ask[og_stock] + 0.02): #was -positionTotal
                exchange.insert_order(og_stock, price=best_bid, volume=max_bid_volume, side="bid", order_type="ioc")
                new = exchange.get_positions()
                time.sleep(0.04)
                if current != new[og_stock]:
                    logger.info(f"Hedge. IOC Bid of {new[og_stock] - current} of {og_stock} at {best_bid}")
                    last_bid[og_stock] = best_bid
                    current = new[og_stock]
         
#%%
last_ask = {
    "NVDA_DUAL": 1,
    "SAN_DUAL": 1,
    "NVDA": 1000,
    "SAN": 1000,
    "CSCO": 1000,
    "ING" : 1000,
    "PFE" : 1000
}
last_bid = {
    "NVDA_DUAL": 1000,
    "SAN_DUAL": 1000,
    "NVDA":1,
    "SAN": 1,
    "CSCO": 1,
    "ING" : 1,
    "PFE" : 1
}

bid_price_history = {}
ask_price_history = {}
HISTORY = 10

def manage_orders(stock_pair, positions):
    global last_bid, last_ask
    original_stock, dual_stock = stock_pair
    orig_bid, orig_ask = get_best_prices(original_stock)
    dual_bid, dual_ask = get_best_prices(dual_stock)

    if orig_bid and orig_ask:
        current_position = positions[dual_stock]
        max_bid_volume = max(0, 50 - current_position)
        max_ask_volume = max(0, current_position + 50)

        # Adjust prices slightly to increase the likelihood of order fills
        bid_price = orig_bid + 0.02  # More aggressive bidding - if -x less aggresive 
        ask_price = orig_ask - 0.02  # More aggressive asking- if +x less aggressive

        if max_bid_volume > 0:
            if bid_price is not None and dual_ask is not None:
                if bid_price > dual_ask:
                    bid_price = orig_bid + (orig_bid-dual_ask)*0.8
            result = exchange.insert_order(dual_stock, price=bid_price, volume=max_bid_volume, side='bid', order_type='ioc')
            time.sleep(0.04)
            new_position = exchange.get_positions()[dual_stock]
            if new_position != current_position:
                logger.info(f"IOC bid order filled for {dual_stock}. Updated position from {current_position} to {new_position} at {bid_price}")
                last_bid[dual_stock] = orig_bid
                positions[dual_stock] = new_position
        
        if current_position != positions[dual_stock]:
            current_position = positions[dual_stock]
            max_bid_volume = max(0, 50 - current_position)
            max_ask_volume = max(0, current_position + 50)

        if max_ask_volume > 0:
            if ask_price is not None and dual_bid is not None:
                if ask_price < dual_bid:
                    ask_price = orig_ask - (dual_bid-ask_price)*0.8
            result = exchange.insert_order(dual_stock, price=ask_price, volume=max_ask_volume, side='ask', order_type='ioc')
            time.sleep(0.04)
            new_position = exchange.get_positions()[dual_stock]
            if new_position != current_position:
                logger.info(f"IOC ask order filled for {dual_stock}. Updated position from {current_position} to {new_position} at {ask_price}")
                last_ask[dual_stock] = orig_ask
                positions[dual_stock] = new_position

    hedge(stock_pair, last_bid, last_ask)
    hedge(('CSCO', ''), last_bid, last_ask)
    hedge(('PFE', ''), last_bid, last_ask)
    hedge(('ING', ''), last_bid, last_ask)

# %%

#Close all positions
def closes_all():
# Get the current open positions
    positions = exchange.get_positions()
    # Iterate over each position
    for stock, quantity in positions.items():
        if quantity != 0:
            print("Current position in", stock, ":", quantity)

            # Determine the side of the order to neutralize the position
            if quantity > 0:
                # If the position is positive, we need to sell to close it
                side = 'ask'  # 'ask' indicates selling
            else:
                # If the position is negative, we need to buy to close it
                side = 'bid'  # 'bid' indicates buying

            # Get the current best price to place the order
            book = exchange.get_last_price_book(stock)
            if side == 'ask':
                price = book.bids[0].price if book.bids else None  # Best bid price
            else:
                price = book.asks[0].price if book.asks else None  # Best ask price

            # If a valid price is found, place the order
            if price:
                volume = abs(quantity)  # The volume should be the absolute value of the quantity
                order_type = 'ioc'  # Immediate-or-cancel to quickly close the position
                result = exchange.insert_order(stock, price=price, volume=volume, side=side, order_type=order_type)
                if result.success:
                    print(f"Successfully placed order to close position in {stock}.")
                else:
                    print(f"Failed to place order for {stock}: {result.error_reason}")
            else:
                print(f"No valid price found for {stock}, cannot place order.")

# Initialize dictionary to track positions dynamically
def initialize_positions1():
    positions = {}
    for og_stock, dual_stock in stock_pairs:
        positions[dual_stock] = exchange.get_positions()[dual_stock]
        positions[og_stock] = exchange.get_positions()[og_stock]

    return positions
initialize_positions1()

#%%

def run_strategies():
    positions = initialize_positions()
    counter = 0
    while True:
        counter+=1
        if counter % 1500 == 0:
            closes_all()
            positions = initialize_positions()
        for pair in stock_pairs:
            manage_orders(pair, positions)
            #arbitraging(pair)
if __name__ == "__main__":
    run_strategies()


#%%